# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234257402960                   -0.50    8.0         
  2   -7.250306401201       -1.79       -1.40    1.0   13.7ms
  3   -7.251178527403       -3.06       -2.11    1.0   12.9ms
  4   -7.251294657912       -3.94       -2.20    2.0   16.8ms
  5   -7.251336213163       -4.38       -2.85    1.0   14.5ms
  6   -7.251337634760       -5.85       -3.11    2.0   16.9ms
  7   -7.251338570873       -6.03       -3.46    1.0   16.0ms
  8   -7.251338784727       -6.67       -4.03    1.0   14.4ms
  9   -7.251338797162       -7.91       -4.55    1.0   14.7ms
 10   -7.251338798599       -8.84       -4.92    2.0   17.0ms
 11   -7.251338798688      -10.05       -5.57    1.0   15.8ms
 12   -7.251338798698      -11.03       -5.51    3.0   19.2ms
 13   -7.251338798703      -11.26       -6.07    1.0   14.7ms
 14   -7.251338798704      -12.05       -6.17    3.0   21.0ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03138490359375267
[ Info: Arnoldi iteration step 2: normres = 0.49150925413628044
[ Info: Arnoldi iteration step 3: normres = 0.8656800419522814
[ Info: Arnoldi iteration step 4: normres = 0.37678892208401576
[ Info: Arnoldi iteration step 5: normres = 0.5661534113911141
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (9.49e-02, 8.31e-02, 5.08e-01, 2.14e-01, 2.06e-02)
[ Info: Arnoldi iteration step 6: normres = 0.27595608823884915
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.20e-02, 1.40e-01, 1.91e-01, 1.08e-01, 4.40e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09253487092473002
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.02e-03, 1.38e-02, 1.42e-02, 6.74e-02, 5.20e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11441338973765317
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.02